In [1]:
import pandas as pd
from pandas import DataFrame
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader, random_split
# from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import math
import numpy as np

In [2]:
# # Import Kaggle data
# import kaggle
# kagggle competitions download tabular-playground-series-mar-2021

In [3]:
# Add data to memory for discovery
train,test = pd.read_csv('./train.csv',index_col="id"),pd.read_csv('./test.csv',index_col="id")

In [15]:
# Get data structure
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300000 entries, 0 to 499999
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cat0    300000 non-null  object 
 1   cat1    300000 non-null  object 
 2   cat2    300000 non-null  object 
 3   cat3    300000 non-null  object 
 4   cat4    300000 non-null  object 
 5   cat5    300000 non-null  object 
 6   cat6    300000 non-null  object 
 7   cat7    300000 non-null  object 
 8   cat8    300000 non-null  object 
 9   cat9    300000 non-null  object 
 10  cat10   300000 non-null  object 
 11  cat11   300000 non-null  object 
 12  cat12   300000 non-null  object 
 13  cat13   300000 non-null  object 
 14  cat14   300000 non-null  object 
 15  cat15   300000 non-null  object 
 16  cat16   300000 non-null  object 
 17  cat17   300000 non-null  object 
 18  cat18   300000 non-null  object 
 19  cont0   300000 non-null  float64
 20  cont1   300000 non-null  float64
 21  cont2   30

In [16]:
# Peek at data examples
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
id,,,,,,,,,,,,,,,,,,,,,
0,A,I,A,B,B,BI,A,S,Q,A,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,A,I,A,A,E,BI,K,W,AD,F,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,A,K,A,A,E,BI,A,E,BM,L,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,A,K,A,C,E,BI,A,Y,AD,F,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,A,I,G,B,E,BI,C,G,Q,A,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [17]:
# Get data structure
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 5 to 499998
Data columns (total 30 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cat0    200000 non-null  object 
 1   cat1    200000 non-null  object 
 2   cat2    200000 non-null  object 
 3   cat3    200000 non-null  object 
 4   cat4    200000 non-null  object 
 5   cat5    200000 non-null  object 
 6   cat6    200000 non-null  object 
 7   cat7    200000 non-null  object 
 8   cat8    200000 non-null  object 
 9   cat9    200000 non-null  object 
 10  cat10   200000 non-null  object 
 11  cat11   200000 non-null  object 
 12  cat12   200000 non-null  object 
 13  cat13   200000 non-null  object 
 14  cat14   200000 non-null  object 
 15  cat15   200000 non-null  object 
 16  cat16   200000 non-null  object 
 17  cat17   200000 non-null  object 
 18  cat18   200000 non-null  object 
 19  cont0   200000 non-null  float64
 20  cont1   200000 non-null  float64
 21  cont2   20

In [18]:
# Peek at data examples
test.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
id,,,,,,,,,,,,,,,,,,,,,
5,A,F,A,A,F,BI,A,AH,AX,A,...,0.735690,0.578366,0.723154,0.228037,0.356227,0.551249,0.655693,0.598331,0.359987,0.947489
6,A,H,C,A,E,AB,I,F,N,A,...,0.313703,0.928885,0.516602,0.600169,0.795224,0.248987,0.654614,0.347944,0.565520,0.388580
8,A,N,C,A,F,AB,A,AH,BC,A,...,0.448201,0.424876,0.344729,0.242073,0.270632,0.746740,0.335590,0.341238,0.252289,0.411592
9,B,L,C,A,F,BI,A,E,AX,A,...,0.666092,0.598943,0.561971,0.806347,0.735983,0.538724,0.381566,0.481660,0.348514,0.325723
11,A,F,A,B,F,BI,A,AH,I,A,...,0.772229,0.479572,0.767745,0.252454,0.354810,0.178920,0.763479,0.562491,0.466261,0.585781


In [6]:
# Get number of unique categories
cal_cols = train.select_dtypes('object')
cal_cols_keys = cal_cols.keys()
# cal_cols_vals = np.array((cal_cols.values))

#Build sample item
cat_stats = np.array([train[c].nunique() for c in cal_cols])
median_cat = math.ceil(np.median(cat_stats))
target_cat_index = int(np.where(cat_stats == median_cat)[0])
target_cat_name = cal_cols_keys[target_cat_index]

print("Categroy with median total unique values:\n",target_cat_name,": ",train[target_cat_name].nunique(),"unique values")

Categroy with median total unique values:
 cat3 :  13 unique values


In [7]:
# Index categories
categoricals_to_ix = {}

for c in train.select_dtypes('object'):
    categoricals_to_ix[c] = {cat:i for i,cat in enumerate(train[c].unique())}

#Confirm with sample indexing
sample_cat = categoricals_to_ix[target_cat_name]
print(target_cat_name,"indices:\n",sample_cat)



cat3 indices:
 {'B': 0, 'A': 1, 'C': 2, 'D': 3, 'G': 4, 'N': 5, 'H': 6, 'F': 7, 'E': 8, 'K': 9, 'I': 10, 'J': 11, 'L': 12}


In [8]:
# Test embeddings
test_embeds = {}

for cix_key in categoricals_to_ix.keys():
    cix = categoricals_to_ix[cix_key]
    bag_size = len(cix)
    embed_size = math.isqrt(bag_size) #hyperparameter to optimize later
    test_embeds[cix_key] = nn.Embedding(bag_size,embed_size)

#Confirm with sample embedding
target_cat_keys = list(sample_cat.keys())
target_cat_vals = np.array(list(sample_cat.values()))
target_cat_vals_median = math.ceil(np.median(target_cat_vals))
target_cat_keys_sample = target_cat_keys[target_cat_vals_median]
print("Sanple",target_cat_name,"category:\n",target_cat_keys_sample)
    
testor = torch.tensor([categoricals_to_ix[target_cat_name][target_cat_keys_sample]], dtype=torch.long)
cat_keys_sample_hash = test_embeds[target_cat_name](testor)
print("Sanple category embedding:\n",cat_keys_sample_hash)

Sanple cat3 category:
 H
Sanple category embedding:
 tensor([[-0.4446, -0.7857, -1.2230]], grad_fn=<EmbeddingBackward>)


In [9]:
# Define embeddings content (dictionary and translations)
#function to build embeddings library
#function assumes only categorical features are fed in
def BuildEmbedlibrary(features):
    categoricals_to_ix,embeds = {},{}
    
    #Build category index
    for c in features:
        categoricals_to_ix[c] = {cat:i for i,cat in enumerate(train[c].unique())}
    
    #Create embeddings for each feature
    for cix_key in categoricals_to_ix.keys():
        cix = categoricals_to_ix[cix_key]
        bag_size = len(cix)
        embed_size = math.isqrt(bag_size) #hyperparameter to optimize later
        embeds[cix_key] = nn.Embedding(bag_size,embed_size)
    
#         print(embeds[cix_key])
    return categoricals_to_ix,embeds

#function that gets respective embedding for given feature value
def CatToSubvector(category,feature_name,categorical_library,embedding_library,verbose=False):
    if verbose:
#         print(verbose)
#         for info in [categorical_library]:
# #         for info in [category,feature_name,categorical_library,embedding_library]:
#             print(info,"\n")
        print("category: {}; feature_name: {}; categorical_library: {}; embedding_library: {}".format(
            category,
            feature_name,
            categorical_library,
            embedding_library
        ))
#     [print(valu,"\n") for valu in [category,feature_name,categorical_library,embedding_library]]
    raw_idx = torch.tensor([categorical_library[feature_name][category]], dtype=torch.long)
#     if (verbose): print("embedding_library[feature_name]:",embedding_library[feature_name],"\n","raw_idx:",raw_idx)
    emb = embedding_library[feature_name](raw_idx)
#     if (verbose): print("embedding_library[feature_name](raw_idx):",emb,"\n")
#     subvector = torch.Tensor(emb)
#     if (verbose): print("subvector:",subvector)
    
#     return subvector
    return emb

In [10]:
# # Confirm function structure
# test_cat_dict,test_embeddings = BuildEmbedlibrary(train.select_dtypes('object'))
# test_subvec = CatToSubvector(target_cat_keys_sample,target_cat_name,test_cat_dict,test_embeddings)

# # print("Sanple category embedding:\n",test_subvec)

In [19]:
# Define data

#define dataset
class ClaimsDataset(Dataset):
    def __init__(self, csv_path='./train.csv',labelled=False):
        """
        Args:
            raw_data (dataframe): Dataframe with raw featurres and labels.
        """
#         self.categoricals,self.embeddings = {},{}
        raw_data = pd.read_csv(csv_path)
        
#         self.l = len(raw_data)
        self.target = raw_data.pop('target') if labelled else torch.tensor([], dtype=torch.long)
        categorical_features = raw_data.select_dtypes(include='object')
        noncategorical_features = raw_data.select_dtypes(exclude='object')
        
        self.categoricals,self.embeddings = BuildEmbedlibrary(categorical_features)
        
        subvects = DataFrame()
#         for row in categorical_features.values:
#             subsubvects = []
#             for v,c in zip(row,categorical_features.columns):
#                 subsubvects.append(CatToSubvector(v,c,self.categoricals,self.embeddings))
# #             [print("subsubvect size:",ssv.size()) for ssv in subsubvects]
#             subvects.append(torch.cat(subsubvects,1))

        subsubvects = []
        for col in categorical_features.columns:
            cat_col = (categorical_features[col]).tolist()
#             print("cat_col:",cat_col,"\n")
#             cat_emb = cat_col
#             cat_emb = cat_col.apply(lambda x: CatToSubvector(x,col,self.categoricals,self.embeddings,True))
#             if (i%2):# == 0 and i!=0):
#                 print("i:",i)
#                 cat_emb = cat_col.apply(lambda x: CatToSubvector(x,col,self.categoricals,self.embeddings,True))
#                 print("cat_emb:",cat_emb,"\n")
#             else:
#                 cat_emb = cat_col.apply(lambda x: CatToSubvector(x,col,self.categoricals,self.embeddings,False))
            cat_embs = CatToSubvector(cat_col[0],col,self.categoricals,self.embeddings,True)
            for x in cat_col[1:]:
                new_emb = CatToSubvector(x,col,self.categoricals,self.embeddings,True)
                cat_embs = torch.cat((cat_embs,new_emb))
#             cat_emb = [CatToSubvector(x,col,self.categoricals,self.embeddings) for x in cat_col]
#             print(col,"cat_embs.shape:",cat_embs.shape,"\n")
#             cat_emb = torch.cat(tuple(cat_emb))
#             print("type(cat_emb[-10]):",type(cat_emb[-10]),"\n")
#             print("cat_emb[-10]:",cat_emb[-10],"\n")
            subsubvects.append(cat_embs)
            
#         print("tuple(subsubvects):",tuple(subsubvects),"\n")
        subvects = torch.cat(tuple(subsubvects),1)
#         print("subvects[1]:",subvects[1],"\n")
#         print("subvects.shape:",subvects.shape,"\n")


        
        # copy the data 
        df_min_max_scaled = noncategorical_features.copy() 

        # apply normalization techniques by Column 1 
        for col in df_min_max_scaled.columns:
            df_min_max_scaled[col] = (df_min_max_scaled[col] - df_min_max_scaled[col].min()) / (df_min_max_scaled[col].max() - df_min_max_scaled[col].min())

#         noncat_mean = noncategorical_features.mean(1)
#         noncat_std = noncategorical_features.std(1)
        
        
        self.data = torch.cat((
            subvects,
            torch.tensor(noncategorical_features.values)
        ),1)
                
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
            
        return x,y
    
    def __len__(self):
        return len(self.vectors)

In [20]:
def LoadData(batch_size_order=2):
    bs = 2*batch_size_order
    train_set,test_set = ClaimsDataset('./train.csv',labelled=True),ClaimsDataset('./test.csv')
    train_loader,test_loader = DataLoader(claimset,batch_size=bs,shuffle=True)
    
    return trainloader,testloader

In [21]:
trainloader,testloader = LoadData()
# print("cd.categoricals:",cd.categoricals,"\n","cd.embeddings:",cd.embeddings,"\n","cd.vectors.shape:",cd.vectors.shape)
x,y = trainloader[0]
print("x:",x)
print("y:",y)

KeyError: 'BU'